In [0]:
import pandas as pd
from sklearn.utils import shuffle


In [0]:
data=pd.read_csv("train.csv")#Making it Relative Path -Mayank
data_ori=data.copy()


In [0]:
data_real_ori = data.copy()

In [0]:
import matplotlib.pyplot as plt  # For plotting graphs 
from datetime import datetime    # To access datetime 
        
%matplotlib inline

In [100]:
data.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [0]:
def datetime(df):
  from datetime import datetime
  df['datetime'] = pd.to_datetime(df['datetime'])
  df['year'] = df['datetime'].dt.year 
  #df['day'] = df['datetime'].dt.day 
  df['hour'] = df['datetime'].dt.hour
  #df['date'] = df['datetime'].dt.date
  df['Day of Week'] = df['datetime'].apply(lambda time: time.dayofweek)
  return df

In [0]:
data_ori=datetime(data_ori)

In [0]:
data_ori=data_ori.drop('datetime',axis=1)


In [0]:
def normalize(df,columns):
    result=df.copy()
    for feature_name in columns :
        mean_value = df[feature_name].mean()
        std_value = df[feature_name].std()
        result[feature_name] = (df[feature_name] - mean_value) / std_value
        
    return (result, mean_value, std_value)

In [0]:
data_ori, mean_temp, sd_temp=normalize(data_ori,["temp"])
data_ori,mean_atemp,sd_atemp=normalize(data_ori,["atemp"])
data_ori,mean_humidity,sd_humidity=normalize(data_ori,["humidity"])
data_ori,mean_windspeed,sd_windspeed=normalize(data_ori,["windspeed"])



In [106]:
data_ori.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,Day of Week
0,1,0,0,1,-1.333599,-1.092687,0.993167,-1.567682,3,13,16,2011,0,5
1,1,0,0,1,-1.438841,-1.182367,0.941206,-1.567682,8,32,40,2011,1,5
2,1,0,0,1,-1.438841,-1.182367,0.941206,-1.567682,5,27,32,2011,2,5
3,1,0,0,1,-1.333599,-1.092687,0.681399,-1.567682,3,10,13,2011,3,5
4,1,0,0,1,-1.333599,-1.092687,0.681399,-1.567682,0,1,1,2011,4,5


In [0]:
data_ori, mean_temp, sd_temp=normalize(data_ori,["temp"])


In [108]:
mean_temp, mean_humidity, mean_windspeed, mean_atemp

(1.1189428545540056e-14,
 61.88645967297446,
 12.799395406945093,
 23.65508405291192)

In [109]:
sd_temp, sd_humidity, sd_windspeed, sd_atemp

(1.000000000000019, 19.245033277394786, 8.16453732683849, 8.474600626484888)

In [0]:



def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix=column)], axis=1)
        data = data.drop(column, axis=1)
    return data



In [0]:


data_ori = dummy_data(data_ori, ["season","weather","hour"])



In [113]:
data_ori.dtypes

holiday          int64
workingday       int64
temp           float64
atemp          float64
humidity       float64
windspeed      float64
casual           int64
registered       int64
count            int64
year             int64
Day of Week      int64
season_1         uint8
season_2         uint8
season_3         uint8
season_4         uint8
weather_1        uint8
weather_2        uint8
weather_3        uint8
weather_4        uint8
hour_0           uint8
hour_1           uint8
hour_2           uint8
hour_3           uint8
hour_4           uint8
hour_5           uint8
hour_6           uint8
hour_7           uint8
hour_8           uint8
hour_9           uint8
hour_10          uint8
hour_11          uint8
hour_12          uint8
hour_13          uint8
hour_14          uint8
hour_15          uint8
hour_16          uint8
hour_17          uint8
hour_18          uint8
hour_19          uint8
hour_20          uint8
hour_21          uint8
hour_22          uint8
hour_23          uint8
dtype: obje

In [0]:

data=shuffle(data_ori,random_state=1)
nrows = len(data_ori)
training_validation_rows = int(nrows*0.8)
data=data_ori[0:training_validation_rows-1]
test1=data_ori[training_validation_rows:]

In [51]:
data.shape

(8707, 43)

In [52]:
test1.shape

(2178, 43)

In [54]:
data.head()

,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,year,Day of Week,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,0,-1.333599,-1.092687,0.993167,-1.567682,3,13,16,2011,5,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,-1.438841,-1.182367,0.941206,-1.567682,8,32,40,2011,5,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,-1.438841,-1.182367,0.941206,-1.567682,5,27,32,2011,5,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,-1.333599,-1.092687,0.681399,-1.567682,3,10,13,2011,5,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,-1.333599,-1.092687,0.681399,-1.567682,0,1,1,2011,5,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:

def split_valid_test_data(data, fraction=0.2):
    data_y = data["count"]
    
    

    data_x = data.drop(["count","casual","registered"], axis=1)

    

    return data_x.values, data_y

data_x, data_y = split_valid_test_data(data)


In [56]:
data_x.shape

(8707, 40)

In [57]:
data.columns

Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'casual', 'registered', 'count', 'year', 'Day of Week', 'season_1',
       'season_2', 'season_3', 'season_4', 'weather_1', 'weather_2',
       'weather_3', 'weather_4', 'hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'],
      dtype='object')

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.optimizers import Adam,SGD,RMSprop,Adagrad

In [0]:

  
adam = Adam(lr=1e-3, decay=1e-3 / 200)
 
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
  
  
    
   

In [59]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = data_x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='softplus'))

# Compile the network :
NN_model.compile(loss='mean_squared_logarithmic_error', optimizer=adam, metrics=['mean_absolute_error','mean_squared_error',rmse])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               5248      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 170,113
Trainable params: 170,113
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose =1, save_best_only = True, mode ='auto')
early_stop=EarlyStopping(monitor='val_loss',patience=10)
callbacks_list = [checkpoint,early_stop]

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()
  
 
 
  plt.legend()
  plt.show()

In [61]:
history=NN_model.fit(data_x, data_y, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


Train on 6965 samples, validate on 1742 samples
Epoch 1/100
6965/6965 [==============================] - 1s 200us/step - loss: 1.9891 - mean_absolute_error: 111.3076 - mean_squared_error: 24966.5719 - rmse: 111.3076 - val_loss: 2.2684 - val_mean_absolute_error: 218.4471 - val_mean_squared_error: 83652.5885 - val_rmse: 218.4471

Epoch 00001: val_loss improved from inf to 2.26836, saving model to Weights-001--2.26836.hdf5
Epoch 2/100
6965/6965 [==============================] - 1s 139us/step - loss: 1.9480 - mean_absolute_error: 110.8926 - mean_squared_error: 24919.7104 - rmse: 110.8926 - val_loss: 2.2368 - val_mean_absolute_error: 217.6597 - val_mean_squared_error: 83212.3419 - val_rmse: 217.6597

Epoch 00002: val_loss improved from 2.26836 to 2.23679, saving model to Weights-002--2.23679.hdf5
Epoch 3/100
6965/6965 [==============================] - 1s 140us/step - loss: 1.8705 - mean_absolute_error: 108.6417 - mean_squared_error: 23932.1154 - rmse: 108.6417 - val_loss: 1.9479 - val_mea

In [160]:
import pickle
from sklearn.externals import joblib


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [161]:
joblib.dump(NN_model,'NN_model.pkl')

['NN_model.pkl']

In [97]:
data.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [0]:
data['count'].describe()

count    8707.000000
mean      174.409670
std       166.438566
min         1.000000
25%        38.000000
50%       130.000000
75%       259.000000
max       891.000000
Name: count, dtype: float64

In [0]:
test=pd.read_csv("test.csv")
test_original=test.copy()

In [0]:
test.shape

(6493, 9)

In [0]:
test.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed'],
      dtype='object')

In [0]:
test['datetime'] = pd.to_datetime(test['datetime'], errors='coerce')

In [0]:
test['year'] = test['datetime'].dt.year 
#test['month'] = test['datetime'].dt.month 
test['day'] = test['datetime'].dt.day 
test['hour'] = test['datetime'].dt.hour

In [0]:
test=test.drop('datetime',axis=1)

In [0]:
test=normalize(test,["temp","atemp","humidity","windspeed"])

In [0]:
test = dummy_data(test, ["season","weather","hour"])

In [0]:
test.shape

(6493, 40)

In [68]:
test1.columns

Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'casual', 'registered', 'count', 'year', 'Day of Week', 'season_1',
       'season_2', 'season_3', 'season_4', 'weather_1', 'weather_2',
       'weather_3', 'weather_4', 'hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'],
      dtype='object')

In [0]:
rest_working_day=1

In [0]:
def transform_temp(temp):
  
  
  return ((temp-mean_temp)/sd_temp)

In [0]:
def transform_atemp(atemp):
  
  return ((atemp-mean_atemp)/sd_atemp)

In [0]:
def transform_humidity(humidity):
  
  return ((humidity-mean_humidity)/sd_humidity)

In [0]:
def transform_windspeed(windspeed):
   return ((windspeed-mean_windspeed)/sd_windspeed)

In [96]:
data_real_ori.tail(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
10876,2012-12-19 14:00:00,4,0,1,1,17.22,21.210,50,12.9980,33,185,218
10877,2012-12-19 15:00:00,4,0,1,1,17.22,21.210,50,19.0012,28,209,237
10878,2012-12-19 16:00:00,4,0,1,1,17.22,21.210,50,23.9994,37,297,334
10879,2012-12-19 17:00:00,4,0,1,1,16.40,20.455,50,26.0027,26,536,562
10880,2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994,23,546,569
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129
10885,2012-12-19 23:00:00,4,0,1,1,13.12,16.665,66,8.9981,4,84,88


In [0]:
import array

In [0]:
def predict_demand(holiday,workingday,temp,atemp,humidity,windspeed,year,Day_of_Week,season_1,
                   season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,
                   hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,
                   hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23):
  
  predict_rest=[holiday, workingday, transform_temp(temp),transform_atemp(atemp),transform_humidity(humidity),transform_windspeed(windspeed),year,Day_of_Week,season_1,
                   season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,
                   hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,
                   hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23 ]
  
 
  
 
  predict_rest=np.array(predict_rest)
  predict_rest=predict_rest.reshape((1, -1))
  prediction=NN_model.predict(predict_rest)
  return prediction[0][0]
  

In [159]:
predict_demand(0,1,22,22,71,24,2012,6,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0)

626.5452

In [0]:
predict_rest=[0,rest_working_day,transform_temp(22),transform_atemp(22),transform_humidity(71),transform_windspeed(24),2012,6,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]

In [0]:
predict_rest=np.array(predict_rest)

In [0]:
predict_rest=predict_rest.reshape((1, -1))


In [146]:
predict_rest.shape

(1, 40)

In [71]:
test1.head(2)

,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,year,Day of Week,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
8708,0,0,1.192201,1.320996,0.629437,0.514237,1,18,19,2012,6,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8709,0,0,1.192201,1.320996,0.889245,0.514237,7,12,19,2012,6,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
data_ori.dtypes

holiday          int64
workingday       int64
temp           float64
atemp          float64
humidity       float64
windspeed      float64
casual           int64
registered       int64
count            int64
year             int64
Day of Week      int64
season_1         uint8
season_2         uint8
season_3         uint8
season_4         uint8
weather_1        uint8
weather_2        uint8
weather_3        uint8
weather_4        uint8
hour_0           uint8
hour_1           uint8
hour_2           uint8
hour_3           uint8
hour_4           uint8
hour_5           uint8
hour_6           uint8
hour_7           uint8
hour_8           uint8
hour_9           uint8
hour_10          uint8
hour_11          uint8
hour_12          uint8
hour_13          uint8
hour_14          uint8
hour_15          uint8
hour_16          uint8
hour_17          uint8
hour_18          uint8
hour_19          uint8
hour_20          uint8
hour_21          uint8
hour_22          uint8
hour_23          uint8
dtype: obje

In [0]:


test1_x, test1_y = split_valid_test_data(test1)

In [67]:
test1_x

array([[ 0.        ,  0.        ,  1.19220087, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.19220087, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.29744255, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.        , -0.80739104, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        , -0.80739104, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  1.        , -0.91263272, ...,  0.        ,
         0.        ,  1.        ]])

In [63]:
scores=NN_model.evaluate(test1_x,test1_y)


2178/2178 [==============================] - 0s 40us/step


In [64]:
print(NN_model.metrics_names)

['loss', 'mean_absolute_error', 'mean_squared_error', 'rmse']


In [0]:
test1_loss=scores[0]
test1_mae=scores[1]
test1_mse=scores[2]
test1_rmse=scores[3]

In [66]:
test1_mae

131.669765861238

In [0]:
#making predictions
prediction=NN_model.predict(predict_rest)

In [150]:
prediction[0][0]

626.5452

In [0]:

evaluation=test_original['datetime'].to_frame()
evaluation["count"]=prediction
evaluation[:10]

,datetime,count
0,2011-01-20 00:00:00,12.484183
1,2011-01-20 01:00:00,5.482817
2,2011-01-20 02:00:00,3.354281
3,2011-01-20 03:00:00,2.179120
4,2011-01-20 04:00:00,2.121344
5,2011-01-20 05:00:00,10.160122
6,2011-01-20 06:00:00,55.021420
7,2011-01-20 07:00:00,150.138443
8,2011-01-20 08:00:00,280.271149
9,2011-01-20 09:00:00,132.132675


In [0]:
evaluation["count"] = evaluation["count"].astype(int)
evaluation["count"]

0        14
1         7
2         4
3         2
4         2
5        13
6        47
7       152
8       262
9       136
10       69
11       74
12      107
13       86
14       79
15       84
16      119
17      259
18      203
19      141
20       91
21       68
22       49
23       32
24       12
25        5
26        3
27        2
28        2
29       12
       ... 
6463     99
6464    115
6465     59
6466     51
6467     38
6468     29
6469     13
6470      6
6471      3
6472      2
6473      2
6474     12
6475     45
6476    133
6477    223
6478    131
6479     61
6480     66
6481     80
6482     86
6483     78
6484     85
6485    110
6486    244
6487    222
6488    145
6489    103
6490     90
6491     73
6492     42
Name: count, Length: 6493, dtype: int64

In [0]:
evaluation.to_csv("submissionv19.csv",index=False)